# Advanced Automation 2025/26 - Assignment 1

To be delivered until 2025-12-05 23:59:59.

**Submission Notes**:
- Create a folder in your group's GitHub repository to solve this assignment. Copy this notebook into that folder.
- You should commit regularly to your repository the answers to the questions in this notebook. If you do not, your grade will be penalized by 1/20 points.
- After running the entire notebook (including graphs and outputs), save the notebook as a .pdf file, by going to File - Print - Destination: Save as PDF.
- Create a .zip file containing both the .ipynb file (the notebook itself) and the .pdf and submit it in Fénix.

# Problem description

The dataset for this assignment contains information on students from two Portuguese secondary schools enrolled in a mathematics course.
Each row corresponds to one student and includes demographic and family background variables (e.g., basic personal and household characteristics), school-related variables (e.g., school context, study habits, past failures), behavioral and lifestyle indicators (e.g., free time, going out, alcohol use), and the grades for the three school periods, on a 0–20 scale.

The goal is to predict the performance of the students in the final period.
First, the problem will be approached as a classification task, predicting whether a student will pass (G3 >= 10) or fail (G3 < 10) the course.
Then, the problem will be approached as a regression task, predicting the actual final grade (G3).

A detailed description of all variables (family context, behavioral, school performance, etc.) is provided in the file `variable_description.txt`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set pandas display options for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported and display options set.")

# Part 1 - Data exploration **(4.00)**

##### **1.1.** Load the dataset from the CSV file `mathematics_grades.csv`. **(0.25)**

In [ ]:
df=pd.read_csv('mathematics_grades.csv')
df.head()

##### **1.2.** Identify the feature types of each variable in the dataset. **(1.50)**

In [ ]:
print("DataFrame Info:")
df.info()

print("\nDataFrame dtypes:")
print(df.dtypes)

# Separating numerical and categorical columns for a clearer view
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()

print("\nNumerical Columns:")
print(numerical_cols)

print("\nCategorical Columns:")
print(categorical_cols)

##### **1.3** Check if the dataset has missing values. If so, discard any row that has a missing value. **(0.25)**

In [ ]:
print("Missing values before handling:")
print(df.isnull().sum())

# Check if there are any missing values
if df.isnull().sum().sum() > 0:
    print("\nMissing values detected. Dropping rows with missing values...")
    initial_rows = df.shape[0]
    df.dropna(inplace=True)
    rows_after_dropping = df.shape[0]
    print(f"Dropped {initial_rows - rows_after_dropping} rows with missing values.")
else:
    print("\nNo missing values detected in the dataset.")

print("\nMissing values after handling:")
print(df.isnull().sum())
print(f"\nDataFrame shape after handling missing values: {df.shape}")

##### **1.4.** Make a pairplot of the numerical variables, colored by the `failed` variable. **(0.50)**

In [ ]:
# Select only numerical columns for the pairplot, and include the 'failed' column for coloring
plot_df = df[numerical_cols + ['failed']]

sns.pairplot(plot_df, hue='failed', palette='coolwarm')
plt.suptitle('Pairplot of Numerical Variables colored by Failed Status', y=1.02) # Adjust suptitle position
plt.show()

##### **1.5.** Make bar plots for each categorical variable, showing the counts for each category, colored by the `failed` variable. **(0.50)**

In [ ]:
for col in categorical_cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=col, hue='failed', palette='viridis')
    plt.title(f'Counts of {col} by Failed Status')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

##### **1.6.** In each school, how many students live in a rural area and do not have internet access at home? **(0.50)**

In [ ]:
# Filter for students living in a rural area ('R') and without internet access ('no')
rural_no_internet_students = df[(df['address'] == 'R') & (df['internet'] == 'no')]

# Group by 'school' and count the number of students
counts_by_school = rural_no_internet_students['school'].value_counts().reset_index()
counts_by_school.columns = ['School', 'Number of Students']

print("Number of students living in a rural area and without internet access, per school:")
print(counts_by_school)

##### **1.7.** Split the dataset into a training set (80%) and a test set (20%). **(0.50)**

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('failed', axis=1)
y = df['failed']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Original DataFrame shape: {df.shape}")
print(f"Training set features shape: {X_train.shape}")
print(f"Testing set features shape: {X_test.shape}")
print(f"Training set target shape: {y_train.shape}")
print(f"Testing set target shape: {y_test.shape}")

print("\nDistribution of 'failed' in original data:")
print(y.value_counts(normalize=True))
print("\nDistribution of 'failed' in training set:")
print(y_train.value_counts(normalize=True))
print("\nDistribution of 'failed' in test set:")
print(y_test.value_counts(normalize=True))

# Part 2 - Classification **(7.50)**

##### You'll start by approaching the problem as a classification task, predicting whether a student will pass (G3 >= 10) or fail (G3 < 10) the course. Consider the variable `failed` as the target variable.

##### **2.1.** Which performance metric do you think are most important for this prediction task, and why? Justify your answer in the context of supporting students who may struggle academically. **(2.00)**

In [ ]:
with open('feature_descriptions.txt', 'r') as f:
    feature_descriptions = f.read()
print(feature_descriptions)

A métrica mais importante é a sensibilidade, ou taxa de verdadeiros positivos. Isto porque ao identificar corretamente alunos que chumbem os apoios são fornecidos a estes com mais necessidades. Reduzimos também os falsos negativos que neste caso representam alunos que segundo o modelo iriam passar mas na verdade chumbam. Portanto neste caso ao termos uma sensibilidade alta indica que o modelo identifica corretamente os alunos que iram chumbar. E isto é crucial para estes casos visto que é um erro mais crítico falhar a identificar alguém que chumbe, do que identificar um aluno que passe como alguém que chumbe.

##### **2.2.** Build a logistic regression model to predict whether a student will pass or fail the course, using as predictor the feature `absences`. Evaluate the performance of the model on the test set using a confusion matrix, accuracy, precision, recall, f1-score and the metric you selected in question 2.1. **(0.50)**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Prepare the feature (absences) for training
# Reshape X_train_absences and X_test_absences to be 2D arrays as expected by sklearn models
X_train_absences = X_train[['absences']]
X_test_absences = X_test[['absences']]

# Initialize and train the Logistic Regression model
# Using 'liblinear' solver for small datasets and 'balanced' class_weight to handle potential imbalance
model_absences = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)
model_absences.fit(X_train_absences, y_train)

# Make predictions on the test set
y_pred_absences = model_absences.predict(X_test_absences)

# Evaluate the model performance
print("\n--- Model Performance Evaluation (using 'absences') ---")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_absences)
print("\nConfusion Matrix:")
print(cm)

# Display the Confusion Matrix visually
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Pass (0)', 'Fail (1)'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Absences Predictor")
plt.show()

# Accuracy
accuracy = accuracy_score(y_test, y_pred_absences)
print(f"\nAccuracy: {accuracy:.4f}")

# Precision (for the 'failed' class, which is True or 1)
precision = precision_score(y_test, y_pred_absences, pos_label=True)
print(f"Precision: {precision:.4f}")

# Recall (for the 'failed' class, which is True or 1) - Metric selected in 2.1
recall = recall_score(y_test, y_pred_absences, pos_label=True)
print(f"Recall (Selected Metric): {recall:.4f}")

# F1-score
f1 = f1_score(y_test, y_pred_absences, pos_label=True)
print(f"F1-score: {f1:.4f}")

print("\nInterpretation of Confusion Matrix:")
print(f"  True Negatives (Correctly predicted as passing): {cm[0, 0]}")
print(f"  False Positives (Predicted as failing, but actually passed): {cm[0, 1]}")
print(f"  False Negatives (Predicted as passing, but actually failed): {cm[1, 0]}")
print(f"  True Positives (Correctly predicted as failing): {cm[1, 1]}")

##### **2.3.** The model you built in question 2.2 likely does not perform very well. Explain why this is the case, considering the distribution of the predictor and the response. How can you improve the model? **(2.00)**

O fraco desempenho do modelo deve-se fundamentalmente à limitação do preditor único (absences), o qual é insuficiente para prever o insucesso escolar, um resultado complexo influenciado por múltiplos fatores como notas anteriores e apoio familiar. A Regressão Logística é incapaz de modelar eficazmente esta relação fraca e potencialmente não linear. Consequentemente, o modelo exibe um baixo Recall de 0.4211, falhando em identificar a maioria dos alunos que realmente chumbam, o que se traduz num elevado número de Falsos Negativos. Para melhorar a performance, é crucial incluir mais características preditivas (como G1, G2, failures e studytime), tratar corretamente as variáveis categóricas, e migrar para modelos mais robustos (como Random Forests ou Gradient Boosting Machines), ajustando-se, por fim, o limiar de classificação para priorizar o Recall.

##### **2.4.** Consider now the variable `studytime` as the predictor and assume it is numerical. Build a logistic regression model to predict whether a student will pass or fail the course. Evaluate the performance of the model on the test set using a confusion matrix, accuracy, precision, recall, f1-score and the metric you selected in question 2.1. **(0.50)**

Note: Use `sklearn.linear_model.LogisticRegression(class_weight='balanced')` for your logistic regression model.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Prepare the feature (studytime) for training, assuming it is numerical
X_train_studytime_numerical = X_train[['studytime']]
X_test_studytime_numerical = X_test[['studytime']]

# Initialize and train the Logistic Regression model with balanced class weights
model_studytime_numerical = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)
model_studytime_numerical.fit(X_train_studytime_numerical, y_train)

# Make predictions on the test set
y_pred_studytime_numerical = model_studytime_numerical.predict(X_test_studytime_numerical)

# Evaluate the model performance
print("\n--- Model Performance Evaluation (using numerical 'studytime') ---")

# Confusion Matrix
cm_num = confusion_matrix(y_test, y_pred_studytime_numerical)
print("\nConfusion Matrix:")
print(cm_num)

# Display the Confusion Matrix visually
disp_num = ConfusionMatrixDisplay(confusion_matrix=cm_num, display_labels=['Pass (0)', 'Fail (1)'])
disp_num.plot(cmap=plt.cm.Greens)
plt.title("Confusion Matrix - Numerical Studytime Predictor")
plt.show()

# Accuracy
accuracy_num = accuracy_score(y_test, y_pred_studytime_numerical)
print(f"\nAccuracy: {accuracy_num:.4f}")

# Precision
precision_num = precision_score(y_test, y_pred_studytime_numerical, pos_label=True)
print(f"Precision: {precision_num:.4f}")

# Recall (for the 'failed' class, which is True or 1) - Metric selected in 2.1
recall_num = recall_score(y_test, y_pred_studytime_numerical, pos_label=True)
print(f"Recall (Selected Metric): {recall_num:.4f}")

# F1-score
f1_num = f1_score(y_test, y_pred_studytime_numerical, pos_label=True)
print(f"F1-score: {f1_num:.4f}")

print("\nInterpretation of Confusion Matrix:")
print(f"  True Negatives (Correctly predicted as passing): {cm_num[0, 0]}")
print(f"  False Positives (Predicted as failing, but actually passed): {cm_num[0, 1]}")
print(f"  False Negatives (Predicted as passing, but actually failed): {cm_num[1, 0]}")
print(f"  True Positives (Correctly predicted as failing): {cm_num[1, 1]}")

##### **2.5.** Repeat question 2.4, but now considering `studytime` as a categorical variable. **(0.50)**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Prepare the feature (studytime) for training, treating it as categorical
# Use OneHotEncoder to convert 'studytime' into categorical features

# Create a OneHotEncoder instance
# handle_unknown='ignore' will allow the encoder to ignore categories not seen during fit
# sparse_output=False ensures a dense array output
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder on the training data and transform both train and test data
X_train_studytime_categorical = encoder.fit_transform(X_train[['studytime']])
X_test_studytime_categorical = encoder.transform(X_test[['studytime']])

# Create DataFrames with column names for better readability (optional, but good practice)
# Get feature names from the encoder
feature_names = encoder.get_feature_names_out(['studytime'])
X_train_studytime_categorical_df = pd.DataFrame(X_train_studytime_categorical, columns=feature_names, index=X_train.index)
X_test_studytime_categorical_df = pd.DataFrame(X_test_studytime_categorical, columns=feature_names, index=X_test.index)


# Initialize and train the Logistic Regression model with balanced class weights
model_studytime_categorical = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)
model_studytime_categorical.fit(X_train_studytime_categorical_df, y_train)

# Make predictions on the test set
y_pred_studytime_categorical = model_studytime_categorical.predict(X_test_studytime_categorical_df)

# Evaluate the model performance
print("\n--- Model Performance Evaluation (using categorical 'studytime') ---")

# Confusion Matrix
cm_cat = confusion_matrix(y_test, y_pred_studytime_categorical)
print("\nConfusion Matrix:")
print(cm_cat)

# Display the Confusion Matrix visually
disp_cat = ConfusionMatrixDisplay(confusion_matrix=cm_cat, display_labels=['Pass (0)', 'Fail (1)'])
disp_cat.plot(cmap=plt.cm.Purples)
plt.title("Confusion Matrix - Categorical Studytime Predictor")
plt.show()

# Accuracy
accuracy_cat = accuracy_score(y_test, y_pred_studytime_categorical)
print(f"\nAccuracy: {accuracy_cat:.4f}")

# Precision
precision_cat = precision_score(y_test, y_pred_studytime_categorical, pos_label=True)
print(f"Precision: {precision_cat:.4f}")

# Recall (for the 'failed' class, which is True or 1) - Metric selected in 2.1
recall_cat = recall_score(y_test, y_pred_studytime_categorical, pos_label=True)
print(f"Recall (Selected Metric): {recall_cat:.4f}")

# F1-score
f1_cat = f1_score(y_test, y_pred_studytime_categorical, pos_label=True)
print(f"F1-score: {f1_cat:.4f}")

print("\nInterpretation of Confusion Matrix:")
print(f"  True Negatives (Correctly predicted as passing): {cm_cat[0, 0]}")
print(f"  False Positives (Predicted as failing, but actually passed): {cm_cat[0, 1]}")
print(f"  False Negatives (Predicted as passing, but actually failed): {cm_cat[1, 0]}")
print(f"  True Positives (Correctly predicted as failing): {cm_cat[1, 1]}")

##### **2.6.** What is the difference between the two models you built in questions 2.4 and 2.5? Plot the predictions of each model over the range of `studytime` values in the test set. What are the advantages and disadvantages of each approach? **(2.00)**


In [ ]:

# --- 1. Preparar os dados para previsão (Range 1 a 4) ---
study_vals = [1, 2, 3, 4]

# Preparar input para o Modelo Numérico (espera 1 coluna numérica)
X_pred_num = pd.DataFrame({'studytime': study_vals})

# Preparar input para o Modelo Categórico (espera variáveis dummy)
# Assumindo que usou drop_first=True, o nível 1 é a base (tudo 0)
# As colunas devem corresponder exatamente às do treino (ex: studytime_2, etc.)
X_pred_cat = pd.DataFrame({
    'studytime_2': [0, 1, 0, 0],
    'studytime_3': [0, 0, 1, 0],
    'studytime_4': [0, 0, 0, 1]
})

# --- 2. Obter as Probabilidades de 'Failed' (classe 1) ---
probs_num = model_num.predict_proba(X_pred_num)[:, 1]
probs_cat = model_cat.predict_proba(X_pred_cat)[:, 1]

# --- 3. Plotar o Gráfico Comparativo ---
plt.figure(figsize=(10, 6))

# Linha do Modelo Numérico (tendência suave)
plt.plot(study_vals, probs_num, marker='o', linestyle='-', color='blue', label='Modelo Numérico (2.4)', linewidth=2)

# Linha do Modelo Categórico (pode ser "quebrada")
plt.plot(study_vals, probs_cat, marker='s', linestyle='--', color='red', label='Modelo Categórico (2.5)', linewidth=2)

plt.title('Model Comparison: Probability of Failure vs. Study Time')
plt.xlabel('Studytime (1 to 4)')
plt.ylabel('Probability of "Failed" (G3 < 10)')
plt.xticks(study_vals, ['1 (<2h)', '2 (2-5h)', '3 (5-10h)', '4 (>10h)'])
plt.ylim(0, 1) # Probabilidade varia entre 0 e 1
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

Modelo 2.4: studytime Numérico (Contínuo)
Tratamento: Assume uma relação linear e contínua entre o tempo de estudo e a probabilidade de falha.

Performance: Apresentou um Recall extremamente baixo (0.2105) e 15 Falsos Negativos (FN). Isso significa que o modelo falhou drasticamente em identificar a maioria dos alunos que realmente chumbaram, considerando o impacto contínuo de studytime como irrelevante e resultando numa curva de previsão muito plana.

Vantagem/Desvantagem: É mais simples, mas a suposição de linearidade revelou-se inadequada para este dataset, comprometendo a capacidade de identificar alunos em risco.

Modelo 2.5: studytime Categórico (One-Hot Encoding)
Tratamento: Atribui um efeito distinto e não linear a cada categoria de tempo de estudo (1, 2, 3, 4).

Performance: Demonstrou um Recall drasticamente superior (0.9474) e apenas 1 Falso Negativo (FN), identificando corretamente quase todos os alunos que falharam. No entanto, a Precision é mais baixa (0.3529), resultando em mais Falsos Positivos (FP).

Vantagem/Desvantagem: É mais flexível e capaz de capturar a verdadeira relação discreta entre os grupos de estudo e a falha, mas é mais complexo e gera mais Falsos Positivos.

Podemos concluir, que para o objetivo de identificar proactivamente alunos que irão chumbar (priorizando o Recall), o Modelo Categórico é claramente superior. A relação entre studytime e a probabilidade de falha não é linear, e o modelo categórico consegue capturar as diferenças de risco inerentes a cada nível de tempo de estudo. Embora gere mais Falsos Positivos (prever falha para quem passa), minimizar os Falsos Negativos (alunos em risco perdidos) é o fator crucial neste contexto de intervenção.

# Part 3 - Regression (3.50)

##### In this part of the assignment, you'll approach the problem as a regression task, predicting the actual final grade `G3`. Use the same training and test sets you created in Part 1.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


##### **3.1.** Build a linear regression model to predict the final grade `G3`, using as predictor the feature `G2`. Evaluate the performance of the model on the test set using Mean Squared Error (MSE) and R-squared ($R^2$). **(0.50)**

In [ ]:
# Define features (X) and target (y) for this specific task
X_train_G2 = X_train[['G2']]
X_test_G2 = X_test[['G2']]
y_train_G3 = X_train['G3']
y_test_G3 = X_test['G3']

# Initialize and train the Linear Regression model
linear_model_G2 = LinearRegression()
linear_model_G2.fit(X_train_G2, y_train_G3)

# Make predictions on the test set
y_pred_G3_G2 = linear_model_G2.predict(X_test_G2)

# Evaluate the model performance
mse_g2 = mean_squared_error(y_test_G3, y_pred_G3_G2)
r2_g2 = r2_score(y_test_G3, y_pred_G3_G2)

print("--- Model Performance Evaluation (G3 predicted by G2) ---")
print(f"Mean Squared Error (MSE): {mse_g2:.4f}")
print(f"R-squared (R^2): {r2_g2:.4f}")

##### **3.2.** Print the equation of the model you built in question 3.1. Plot the predictions of the model as a function of the inputs, along with the actual data points from the test set. Interpret the coefficients of the model. **(1.00)**


In [ ]:
# Print the equation of the model
intercept = linear_model_G2.intercept_
coefficient = linear_model_G2.coef_[0]
print(f"--- Model Equation (G3 predicted by G2) ---")
print(f"G3 = {coefficient:.4f} * G2 + {intercept:.4f}")

# Plot the predictions and actual data points
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test_G2['G2'], y=y_test_G3, label='Actual G3', color='blue', alpha=0.6)
sns.lineplot(x=X_test_G2['G2'], y=y_pred_G3_G2, label='Predicted G3', color='red', linestyle='--')
plt.title('Actual vs. Predicted G3 based on G2')
plt.xlabel('G2 (Second Period Grade)')
plt.ylabel('G3 (Final Grade)')
plt.grid(True)
plt.legend()
plt.show()

O interceto (0.2559) representa o valor previsto para G3 quando G2 é igual a 0. Contudo, no contexto das notas, uma nota G2 de 0 pode não ter um significado prático ou pode não ser um cenário real para os dados observados, tornando a interpretação literal do interceto, neste caso, menos relevante. O coeficiente de G2 (0.9929) é o mais importante neste modelo, indicando que para cada aumento de uma unidade na nota G2, a nota final predita G3 aumenta em aproximadamente 0.9929 unidades. Isso sugere uma relação quase linear e fortemente positiva entre a nota do segundo período (G2) e a nota final (G3), onde notas G2 mais altas estão associadas a notas G3 mais altas.

##### **3.3.** Build a linear regression model to predict the final grade `G3`, using as predictors all features except the variables related to the grades (`G1`, `G2`, `G3`, `failed`). Explain the choices you made. Evaluate the performance of the model on the test set using Mean Squared Error (MSE) and R-squared ($R^2$). Plot the predicted vs actual values of `G3` for the test set. **(1.00)**

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define the target variable for regression (G3) from the original splits
y_train_G3 = X_train['G3']
y_test_G3 = X_test['G3']

# Define features to exclude from the predictors (grades and the 'failed' target)
features_to_exclude = ['G1', 'G2', 'G3'] # 'failed' is already dropped when X was created

# Create feature sets without the excluded grades
X_train_regression = X_train.drop(columns=features_to_exclude)
X_test_regression = X_test.drop(columns=features_to_exclude)

# Identify categorical and numerical columns for preprocessing
categorical_features = X_train_regression.select_dtypes(include='object').columns
numerical_features = X_train_regression.select_dtypes(include=np.number).columns

# Create a ColumnTransformer to apply OneHotEncoder to categorical features
# and leave numerical features untouched
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # In case there are any other columns not specified
)

# Apply the preprocessing to the training and test sets
X_train_processed = preprocessor.fit_transform(X_train_regression)
X_test_processed = preprocessor.transform(X_test_regression)

# If the output is a sparse matrix, convert to dense for LinearRegression (if needed)
if hasattr(X_train_processed, 'toarray'):
    X_train_processed = X_train_processed.toarray()
if hasattr(X_test_processed, 'toarray'):
    X_test_processed = X_test_processed.toarray()

# Initialize and train the Linear Regression model
linear_model_all_features = LinearRegression()
linear_model_all_features.fit(X_train_processed, y_train_G3)

# Make predictions on the test set
y_pred_G3_all_features = linear_model_all_features.predict(X_test_processed)

# Evaluate the model performance
mse_all = mean_squared_error(y_test_G3, y_pred_G3_all_features)
r2_all = r2_score(y_test_G3, y_pred_G3_all_features)

print("--- Model Performance Evaluation (G3 predicted by all features except grades) ---")
print(f"Mean Squared Error (MSE): {mse_all:.4f}")
print(f"R-squared (R^2): {r2_all:.4f}")

# Plot predicted vs actual values
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test_G3, y=y_pred_G3_all_features)
plt.plot([y_test_G3.min(), y_test_G3.max()], [y_test_G3.min(), y_test_G3.max()], 'r--', lw=2, label='Perfect Prediction')
plt.title('Predicted vs. Actual G3 (All features except grades)')
plt.xlabel('Actual G3')
plt.ylabel('Predicted G3')
plt.grid(True)
plt.legend()
plt.show()

##### **3.4.** In terms of application, what is the difference between building a model to predict student performance using the grades of the previous periods (as in question 3.1) versus using other features (as in question 3.3)? When would each approach be more appropriate? **(1.00)**

Usar G2 para prever G3 aproveita a forte correlação entre a nota do segundo período e a nota final, resultando em alta precisão preditiva. Essa abordagem é adequada para previsão de curto prazo ou sistemas de alerta precoce, quando as notas anteriores estão disponíveis. Em contraste, usar todas as outras variáveis não relacionadas a notas, como demográficas, comportamentais, informações escolares, evita o data leakage e permite identificar fatores que influenciam o desempenho além das notas anteriores, embora geralmente tenha menor precisão preditiva. Essa abordagem é mais apropriada para entender os fatores que afetam o desempenho estudantil.

# Part 4 - Cross-validation (4.00)

##### **4.1.** Explain the concept of cross-validation and its importance in evaluating machine learning models. **(1.00)**

A cross-validation é uma metodologia desenvolvida para avaliar como um modelo de machine learning deverá comportar-se quando aplicado a dados novos e nunca antes observados. Em vez de depender de uma única divisão do conjunto de dados em treino e teste, o método divide repetidamente os dados em vários subconjuntos, permitindo que o modelo seja treinado e avaliado múltiplas vezes. Esta avaliação repetida produz uma estimativa mais estável e fiável da capacidade preditiva do modelo. O seu valor reside no facto de reduzir o risco de conclusões enviesadas causadas por uma divisão de treino/teste excepcionalmente favorável ou desfavorável. Assim, a validação cruzada aumenta a confiança na comparação entre modelos, apoia decisões de seleção e afinação de modelos e oferece uma visão mais clara da capacidade de generalização para além da amostra usada originalmente no treino.

##### **4.2.** Describe how K-fold cross-validation works. What values of K should be used? Justify. **(1.00)**

A cross-validation K-fold funciona dividindo o conjunto de dados em K blocos de tamanho aproximadamente igual. O modelo é então treinado K vezes de forma independente. Em cada iteração, um dos blocos é deixado de fora para servir como conjunto de validação, enquanto os restantes K−1 blocos constituem o conjunto de treino. Depois de todos os blocos terem desempenhado o papel de validação uma vez, os K resultados obtidos são promediados, fornecendo uma estimativa global do desempenho do modelo. Este método garante que todas as observações são utilizadas tanto para treino como para validação, mas nunca simultaneamente. Quanto à escolha do valor de K, os mais utilizados na prática são 5 e 10. Estes valores representam um equilíbrio adequado entre ter subconjuntos suficientemente representativos e manter o custo computacional em níveis razoáveis. Valores demasiado baixos tendem a produzir estimativas com maior variância, enquanto valores muito elevados, como o leave-one-out cross-validation, podem ser computacionalmente dispendiosos e gerar estimativas instáveis, uma vez que os conjuntos de treino diferem muito pouco entre iterações.

##### **4.3.** Explain how K-Fold cross-validation should be done to choose the classification threshold for the logistic regression model of Part 2. **(2.00)**

No modelo de regressão logística, a classificação é feita usando o limiar padrão de 0.5, que converte probabilidades previstas em classes. Para definir um limiar mais adequado ao problema e ao conjunto de dados, este valor não deve ser escolhido após treinar o modelo, mas sim determinado dentro de um processo de K-Fold cross-validation aplicado exclusivamente aos dados de treino. A ideia central é avaliar diferentes limiares possíveis enquanto o modelo é validado repetidamente em subconjuntos distintos do próprio treino. Em cada uma das K divisões, o conjunto de treino é dividido em partições com dados internos de treino e de validação. Em seguida, treina-se um modelo de regressão logística com as mesmas configurações. Depois do treino, calculam-se as probabilidades previstas para o subconjunto de validação utilizando model. Estas probabilidades são comparadas com um conjunto de limiares candidatos (por exemplo, de 0.05 até 0.95 com incrementos de 0.05). Para cada limiar, a probabilidade é convertida em classe: prevê-se “fail” (1) quando a probabilidade ultrapassa o limiar e “pass” (0) caso contrário. Para cada limiar são calculadas as métricas de desempenho na validação. A métrica de referência é a sensibilidade, portanto esse será o valor usado para comparar limiares. A sensibilidade obtido em cada fold é registado para cada limiar, e o processo repete-se até completar todos os K folds. No final, para cada limiar obtém-se o valor médio de sensibilidade ao longo das K execuções. O limiar ideal será aquele que apresentar maior sensibilidade. Depois de escolhido esse limiar ótimo, treina-se novamente o modelo final usando todo o conjunto de treino com as mesmas características. Finalmente, as previsões futuras deixam de utilizar o limiar padrão de 0.5 e passam a utilizar o limiar determinado através do processo de K-Fold. Desta forma, a escolha do limiar não utiliza o conjunto de teste, evita data leakage e assegura que o limiar selecionado realmente reflete a capacidade de generalização do modelo.

# Part 5 - GitHub (1.00)

##### **5.1.** Describe how your group used GitHub to track changes throughout the project and explain why maintaining a clear change history is important in data analysis and machine learning work. **(1.00)**

Ao longo do projeto, o grupo utilizou o GitHub como plataforma central para gerir versões e acompanhar a evolução do trabalho. As contribuições individuais foram feitas através de commits regulares, sempre acompanhados de mensagens descritivas que identificavam claramente as alterações realizadas, como a introdução de novos métodos, correções de erros, melhoria de visualizações ou reorganização da estrutura do projeto. Sempre que necessário, foram criados branches separados para desenvolver funcionalidades ou análises específicas sem interferir com a versão principal, assegurando que o processo de integração era organizado e controlado. O uso de pull requests permitiu rever e discutir alterações antes de as incorporar na versão final, garantindo consistência e qualidade no código. Manter um histórico de alterações claro é fundamental em projetos de análise de dados e machine learning porque estes exigem rastreabilidade, reprodutibilidade e controlo rigoroso das diferentes versões dos modelos e dos dados. Um histórico bem estruturado permite compreender a origem de resultados, identificar rapidamente quando e onde surgiram falhas ou divergências e justificar decisões metodológicas tomadas ao longo do desenvolvimento. Além disso, facilita o trabalho colaborativo, evitando conflitos, perdas de código e duplicação de esforços.